In [ ]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import os
import time
import math
from datetime import datetime
# from prophet import Prophet

import boto3
import sagemaker

In [ ]:
# from pytorch_forecasting.data.examples import get_stallion_data
# data = get_stallion_data()  # load data as pandas dataframe

vent = pd.read_csv('data/example/train_OwBvO8W/event_calendar.csv')
historical = pd.read_csv('data/example/train_OwBvO8W/historical_volume.csv')
soda = pd.read_csv('data/example/train_OwBvO8W/industry_soda_sales.csv')
industry = pd.read_csv('data/example/train_OwBvO8W/industry_volume.csv')
price = pd.read_csv('data/example/train_OwBvO8W/price_sales_promotion.csv')
weather = pd.read_csv('data/example/train_OwBvO8W/weather.csv')

data = historical.merge(price,on=['Agency','SKU','YearMonth'],how='left')
data = data.merge(soda,on=['YearMonth'],how='left')
data = data.merge(industry,on='YearMonth',how='left')
data = data.merge(event,on=['YearMonth'],how='left')

print(data.shape)

data.iloc[[291,871,19532],:]

In [ ]:
data.columns = [column_name.lower() for column_name in data.columns]
def create_date_column(x):
    if not x.yearmonth :
        return None
    else:
        return pd.datetime(year=int(x.yearmonth)//100,month=int(x.yearmonth)%100,day=1)
data.loc[:,'date'] = data.apply(lambda x:create_date_column(x), axis=1)
data.columns = [column_name.lower() for column_name in data.columns]
data = data.rename(columns={
                            'price':'price_regular','sales':'price_actual',
                            'promotions':'discount'
                           })
def calculate_discount_percentage(x):
    if x.price_regular == 0:
        return 0
    else:
        return x.discount/ x.price_regular
data.loc[:,'discount_in_percent'] = data.apply(lambda x:calculate_discount_percentage(x), axis=1)

In [ ]:
# add time index
data["time_idx"] = data["date"].dt.year * 12 + data["date"].dt.month
data["time_idx"] -= data["time_idx"].min()
# add additional features
# categories have to be strings
data["month"] = data.date.dt.month.astype(str).astype("category")
data["log_volume"] = np.log(data.volume + 1e-8)
data["avg_volume_by_sku"] = (
    data
    .groupby(["time_idx", "sku"], observed=True)
    .volume.transform("mean")
)
data["avg_volume_by_agency"] = (
    data
    .groupby(["time_idx", "agency"], observed=True)
    .volume.transform("mean")
)
# we want to encode special days as one variable and 
# thus need to first reverse one-hot encoding
special_days = [
    "easter day", "good friday", "new year", "christmas",
    "labor day", "independence day", "revolution day memorial",
    "regional games ", "fifa u-17 world cup", "football gold cup",
    "beer capital", "music fest"
]
data[special_days] = (
    data[special_days]
    .apply(lambda x: x.map({0: "-", 1: x.name}))
    .astype("category")
)
# show sample data
data.sample(10, random_state=521)

In [ ]:
from pytorch_forecasting.data import (
    TimeSeriesDataSet,
    GroupNormalizer
)
max_prediction_length = 6  # forecast 6 months
max_encoder_length = 24  # use 24 months of history
training_cutoff = data["time_idx"].max() - max_prediction_length
training = TimeSeriesDataSet(
    data[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="volume",
    group_ids=["agency", "sku"],
    min_encoder_length=0,  # allow predictions without history
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=["agency", "sku"],
    static_reals=[
#         "avg_population_2017",
#         "avg_yearly_household_income_2017"
    ],
    time_varying_known_categoricals=["special_days", "month"],
    # group of categorical variables can be treated as 
    # one variable
    variable_groups={"special_days": special_days},
    time_varying_known_reals=[
        "time_idx",
        "price_regular",
        "discount_in_percent"
    ],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=[
        "volume",
        "log_volume",
        "industry_volume",
        "soda_volume",
#         "avg_max_temp",
        "avg_volume_by_agency",
        "avg_volume_by_sku",
    ],
    target_normalizer=GroupNormalizer(
        groups=["agency", "sku"]
    ),  # use softplus with beta=1.0 and normalize by group
    add_relative_time_idx=True,  # add as feature
    add_target_scales=True,  # add as feature
    add_encoder_length=True,  # add as feature
)
# create validation set (predict=True) which means to predict the
# last max_prediction_length points in time for each series
validation = TimeSeriesDataSet.from_dataset(
    training, data, predict=True, stop_randomization=True
)
# create dataloaders for model
batch_size = 128
train_dataloader = training.to_dataloader(
    train=True, batch_size=batch_size, num_workers=0
)
val_dataloader = validation.to_dataloader(
    train=False, batch_size=batch_size * 10, num_workers=0
)

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger

from pytorch_forecasting.metrics import QuantileLoss
from pytorch_forecasting.models import TemporalFusionTransformer
# stop training, when loss metric does not improve on validation set
early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=1e-4,
    patience=10,
    verbose=False,
    mode="min"
)
# lr_logger = LearningRateLogger()  # log the learning rate
lr_logger = LearningRateMonitor()
logger = TensorBoardLogger("lightning_logs")  # log to tensorboard
# create trainer
trainer = pl.Trainer(
    max_epochs=30,
    gpus=0,  # train on CPU, use gpus = [0] to run on GPU
    gradient_clip_val=0.1,
#     early_stop_callback=early_stop_callback,
    limit_train_batches=30,  # running validation every 30 batches
    # fast_dev_run=True,  # comment in to quickly check for bugs
    callbacks=[lr_logger],
    logger=logger,
)
# initialise model
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.03,
    hidden_size=16,  # biggest influence network size
    attention_head_size=1,
    dropout=0.1,
    hidden_continuous_size=8,
    output_size=7,  # QuantileLoss has 7 quantiles by default
    loss=QuantileLoss(),
    log_interval=10,  # log example every 10 batches
    reduce_on_plateau_patience=4,  # reduce learning automatically
)
tft.size() # 29.6k parameters in model
# fit network
trainer.fit(
    tft,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader
)

In [ ]:
from pytorch_forecasting.metrics import MAE,MAPE
# load the best model according to the validation loss (given that
# we use early stopping, this is not necessarily the last epoch)
best_model_path = trainer.checkpoint_callback.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)
# calculate mean absolute error on validation set
actuals = torch.cat([y[0] for x, y in val_dataloader])
predictions = best_tft.predict(val_dataloader)
previous = torch.cat([x['encoder_target'] for x, y in val_dataloader])

In [ ]:
index = [11,12,13]
for i in index:
    plt.plot(previous[i],label='previous')
    plt.plot([24+i for i in range(len(predictions[i]))],predictions[i],label='predict')
    plt.plot([24+i for i in range(len(actuals[i]))], actuals[i],label='true')
    plt.legend()
    plt.show()

In [ ]:
def mape(true,pred, basic_reduction = 10, return_mean=False):
    mape = []
    for i in range(0,len(true)):
        pred_val, true_val = pred[i], true[i]
        if pred_val < 0:
            pred_val = 0
        if  true_val < basic_reduction:
            true_val = basic_reduction
        mape.append(abs(pred_val- true_val)/true_val)
    if return_mean:
        return sum(mape)/len(mape)
    else:
        return mape

In [ ]:
from pytorch_forecasting.metrics import SMAPE
# calculate metric by which to display
predictions = best_tft.predict(val_dataloader )
mean_losses = SMAPE(reduction="none")(predictions, actuals).mean(1)
indices = mean_losses.argsort(descending=True)  # sort losses
raw_predictions,x = best_tft.predict(val_dataloader, mode="raw", return_x=True)
# show only two examples for demonstration purposes
for idx in range(5):
    best_tft.plot_prediction(
        x,
        raw_predictions,
        idx=indices[idx],
#         add_loss_to_title=SMAPE()
    )

In [ ]:
interpretation = best_tft.interpret_output(
    raw_predictions, reduction="sum"
)
best_tft.plot_interpretation(interpretation)

In [ ]:
data[(data.agency=='Agency_22') & (data.sku=='SKU_01')]